# Self-attention with trainable weights

**Descrição**  
A *self-attention with trainable weights* é a versão “completa” do mecanismo de atenção usada em Transformers: em vez de comparar diretamente os embeddings dos tokens, aprendemos **três projeções lineares** (pesos treináveis) que transformam cada token em vetores **Query (Q)**, **Key (K)** e **Value (V)**.  
Com isso, o modelo passa a decidir *como* medir similaridade (via Q·K) e *quais informações* combinar (via V), ajustando esses pesos durante o treinamento para produzir representações contextuais mais úteis.

**Objetivo**  
Este notebook tem como objetivo mostrar, de forma didática, como implementar self-attention com parâmetros treináveis em PyTorch, destacando:
- por que introduzimos matrizes/pesos para gerar **Q, K e V** (em vez de usar os embeddings “crus”);
- como esses pesos tornam a atenção **aprendível** (ajustável por backprop);
- como os **pesos de atenção** (dinâmicos, por entrada) diferem dos **pesos do modelo** (parâmetros treináveis das projeções).

**Funcionamento**  
Em alto nível, o mecanismo segue estes passos:

1. **Entrada**: uma sequência de vetores (embeddings) `X` com shape `[seq_len, d_in]`.
2. **Projeções treináveis**: aplica-se três camadas lineares para obter:
   - `Q = W_q(X)`, `K = W_k(X)`, `V = W_v(X)` (cada uma com shape `[seq_len, d_attn]`).
3. **Scores de atenção**: calcula-se a similaridade entre cada query e todas as keys:
   - `scores = Q @ K.T`  → shape `[seq_len, seq_len]`.
4. **Normalização**: aplica-se `softmax` por linha para transformar scores em probabilidades:
   - `attn_weights = softmax(scores, dim=-1)` (cada linha soma 1).
5. **Agregação de contexto**: combina-se a informação dos values com esses pesos:
   - `context = attn_weights @ V` → shape `[seq_len, d_attn]`.
6. **Saída**: o `context` é a representação contextualizada de cada token, agora construída a partir de uma combinação ponderada dos demais tokens — e essa combinação é guiada por **projeções aprendidas** (Wq, Wk, Wv).
 


![O gato fazendo supino](../../imagens/cap03/02_gato_sobe_no_tapete.png)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Para reprodutibilidade
torch.manual_seed(42)

## Frase de exemplo e vocabulário

In [2]:
# Frase de exemplo
sentence = "O gato sobe no tapete".split()

# Vocabulário
vocab = {word: idx for idx, word in enumerate(sentence)}
inv_vocab = {idx: word for word, idx in vocab.items()}

print("Vocabulário:")
for k, v in vocab.items():
    print(f"{k} -> {v}")

Vocabulário:
O -> 0
gato -> 1
sobe -> 2
no -> 3
tapete -> 4


## Conversão para índices (tokens)

In [3]:
# Convertendo palavras para índices
token_ids = torch.tensor([vocab[word] for word in sentence])

print("Tokens:")
for word, idx in zip(sentence, token_ids, strict=False):
    print(f"{word} -> {idx.item()}")

Tokens:
O -> 0
gato -> 1
sobe -> 2
no -> 3
tapete -> 4


## Camada de Embedding

In [4]:
vocab_size = len(vocab)
embedding_dim = 3  # dimensão pequena para fins didáticos

embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# Aplicando embedding
X = embedding(token_ids)

print("Shape dos embeddings:", X.shape)
X

Shape dos embeddings: torch.Size([5, 3])


tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863],
        [ 2.2082, -0.6380,  0.4617],
        [ 0.2674,  0.5349,  0.8094],
        [ 1.1103, -1.6898, -0.9890]], grad_fn=<EmbeddingBackward0>)

## Conceito: o que são Query, Key e Value?

Em **self-attention**, cada token da sequência “conversa” com todos os outros tokens para produzir uma **representação contextualizada**.

- **Query (Q):** representa o que o token está procurando (a “pergunta” do token).
- **Key (K):** representa o que cada token oferece como índice para ser encontrado (as “etiquetas” / “chaves”).
- **Value (V):** representa a informação que será agregada (o “conteúdo” que será combinado).

O mecanismo funciona assim (para cada token *i*):

1. Calcula-se um **score de similaridade** entre o token *i* e cada token *j*:
   
   <img src="https://latex.codecogs.com/svg.image?&space;\text{score}_{i,j}=q_i\cdot&space;k_j"/>

3. Aplica-se **softmax** para transformar scores em pesos (que somam 1):


   <img src="https://latex.codecogs.com/svg.image?\alpha_{i,j}=\text{softmax}(score_{i,:})"/>


3. Forma-se o **context vector** do token *i* como a média ponderada dos values:


    <img src="https://latex.codecogs.com/svg.image?&space;\text{context}_i=\sum_j\alpha_{i,j}v_j&space;"/>



![Attention gato](../../imagens/cap03/02_attention_gato.png)

## Definição da camada de Self-Attention (pesos treináveis)

In [5]:
class SelfAttention(nn.Module):
    """
    Implementa Self-Attention (single-head) para uma sequência, sem batch explícito,
    retornando também tensores intermediários úteis para inspeção/debug.

    Parâmetros:
    ----------
    d_in : int
        Dimensão de entrada (features por token).
    d_attn : int
        Dimensão interna da atenção (projeções Q, K e V).

    Retorno:
    -------
    tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]
        Retorna (Q, K, V, scores, context, attn_weights), onde:
        - Q: (seq_len, d_attn)
        - K: (seq_len, d_attn)
        - V: (seq_len, d_attn)
        - scores: (seq_len, seq_len)
        - context: (seq_len, d_attn)
        - attn_weights: (seq_len, seq_len)

    Exceções:
    --------
    Levanta ValueError se x não tiver shape 2D (seq_len, d_in).
    """

    def __init__(self, d_in: int, d_attn: int) -> None:
        super().__init__()

        self.W_q = nn.Linear(d_in, d_attn, bias=False)
        self.W_k = nn.Linear(d_in, d_attn, bias=False)
        self.W_v = nn.Linear(d_in, d_attn, bias=False)

    def forward(
        self, x: torch.Tensor
    ) -> tuple[
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
    ]:
        """
        Executa self-attention na sequência de entrada.

        Parâmetros:
        ----------
        x : torch.Tensor
            Tensor de shape (seq_len, d_in).

        Retorno:
        -------
        tuple
            (Q, K, V, scores, context, attn_weights)
        """
        if x.ndim != 2:
            raise ValueError(
                f"Esperado x com 2 dimensões (seq_len, d_in), mas veio shape={tuple(x.shape)}."
            )

        # Projeções
        Q = self.W_q(x)  # (T, d_attn)
        K = self.W_k(x)  # (T, d_attn)
        V = self.W_v(x)  # (T, d_attn)

        # Scores: (T, d_attn) @ (d_attn, T) -> (T, T)
        scores = Q @ K.T

        # Pesos de atenção: (T, T)
        attn_weights = F.softmax(scores, dim=-1)

        # Contexto: (T, T) @ (T, d_attn) -> (T, d_attn)
        context = attn_weights @ V

        return Q, K, V, scores, context, attn_weights

### Inicialização e verificação dos pesos

In [6]:
d_attn = 2  # apenas 2 para ficar didático

self_attention = SelfAttention(d_in=embedding_dim, d_attn=d_attn)

Q, K, V, scores, context, attn_weights = self_attention(X)

print("Pesos da query")
print(f"{self_attention.W_q.weight=}")
print()
print("Pesos das keys")
print(f"{self_attention.W_k.weight=}")
print()
print("Pesos dos values")
print(f"{self_attention.W_v.weight=}")

Pesos da query
self_attention.W_q.weight=Parameter containing:
tensor([[ 0.4457,  0.0961, -0.1875],
        [ 0.3568,  0.0900,  0.4665]], requires_grad=True)

Pesos das keys
self_attention.W_k.weight=Parameter containing:
tensor([[ 0.0631, -0.1821,  0.1551],
        [-0.1566,  0.2430,  0.5155]], requires_grad=True)

Pesos dos values
self_attention.W_v.weight=Parameter containing:
tensor([[ 0.3337, -0.2524,  0.3333],
        [ 0.1033,  0.2932, -0.3519]], requires_grad=True)


### O Q do gato

In [7]:
Q[1]

tensor([ 0.0297, -0.1058], grad_fn=<SelectBackward0>)

### O K do gato

In [8]:
K[1]

tensor([ 0.1901, -0.4049], grad_fn=<SelectBackward0>)

### O V do gato

In [9]:
V[1]

tensor([ 0.2982, -0.2399], grad_fn=<SelectBackward0>)

### Scores do gato em relação ao próprio gato

In [10]:
scores[1][1]

tensor(0.0485, grad_fn=<SelectBackward0>)

## Score do gato em relação aos demais itens

In [11]:
scores[1]

tensor([-0.0095,  0.0485,  0.0375, -0.0521,  0.1224],
       grad_fn=<SelectBackward0>)

### Normalização por softmax

In [12]:
attn_weights[1]

tensor([0.1920, 0.2035, 0.2013, 0.1840, 0.2191], grad_fn=<SelectBackward0>)

In [13]:
scores

tensor([[ 0.0280, -0.0751, -0.0246,  0.1272, -0.2372],
        [-0.0095,  0.0485,  0.0375, -0.0521,  0.1224],
        [ 0.1226, -0.2240,  0.0251,  0.5156, -0.8472],
        [ 0.0525, -0.2074, -0.1308,  0.2641, -0.5659],
        [-0.0039,  0.1864,  0.2265, -0.0865,  0.3539]], grad_fn=<MmBackward0>)

In [14]:
attn_weights

tensor([[0.2118, 0.1910, 0.2009, 0.2338, 0.1624],
        [0.1920, 0.2035, 0.2013, 0.1840, 0.2191],
        [0.2235, 0.1580, 0.2027, 0.3311, 0.0847],
        [0.2284, 0.1761, 0.1902, 0.2822, 0.1231],
        [0.1718, 0.2079, 0.2164, 0.1582, 0.2458]], grad_fn=<SoftmaxBackward0>)

In [15]:
context

tensor([[ 0.4301, -0.1011],
        [ 0.4464, -0.1008],
        [ 0.4094, -0.1007],
        [ 0.4094, -0.1000],
        [ 0.4670, -0.1018]], grad_fn=<MmBackward0>)

In [16]:
print("Shape do contexto:", context.shape)
print("Shape dos pesos de atenção:", attn_weights.shape)

Shape do contexto: torch.Size([5, 2])
Shape dos pesos de atenção: torch.Size([5, 5])


## Visualizando os pesos de atenção (parte mais didática)

Aqui mostramos quem presta atenção em quem.

In [17]:
print("Pesos de atenção:\n")

for i, word in enumerate(sentence):
    print(f"Palavra: '{word}'")
    for j, weight in enumerate(attn_weights[i]):
        print(f"  → atenção em '{sentence[j]}': {weight.item():.2f}")
    print()

Pesos de atenção:

Palavra: 'O'
  → atenção em 'O': 0.21
  → atenção em 'gato': 0.19
  → atenção em 'sobe': 0.20
  → atenção em 'no': 0.23
  → atenção em 'tapete': 0.16

Palavra: 'gato'
  → atenção em 'O': 0.19
  → atenção em 'gato': 0.20
  → atenção em 'sobe': 0.20
  → atenção em 'no': 0.18
  → atenção em 'tapete': 0.22

Palavra: 'sobe'
  → atenção em 'O': 0.22
  → atenção em 'gato': 0.16
  → atenção em 'sobe': 0.20
  → atenção em 'no': 0.33
  → atenção em 'tapete': 0.08

Palavra: 'no'
  → atenção em 'O': 0.23
  → atenção em 'gato': 0.18
  → atenção em 'sobe': 0.19
  → atenção em 'no': 0.28
  → atenção em 'tapete': 0.12

Palavra: 'tapete'
  → atenção em 'O': 0.17
  → atenção em 'gato': 0.21
  → atenção em 'sobe': 0.22
  → atenção em 'no': 0.16
  → atenção em 'tapete': 0.25

